<a href="https://colab.research.google.com/github/asraj/END/blob/main/END_session9_model1_CommonsenseQA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

CommonsenseQA is a set of multiple-choice question answer data that requires different types of common sense knowledge to predict the correct answers

## **Data Preparation**
The data is available as json file in Kaggle. The following code

1) downloads the zip file from kaggle

2) convert json file into dataframe

3) get the answer text from the answer choice

4) create separate columns for question and answer and save for future use

In [16]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
!ls /content/gdrive/MyDrive/kaggle

kaggle.json  toc.csv	    tut3-model.pt  tweets.tsv
sample.csv   tut2-model.pt  twcs	   Ubuntu-dialogue-corpus


In [10]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/kaggle"
# /content/gdrive/My Drive/Kaggle is the path where kaggle.json is present in the Google Drive

In [3]:
!pwd

/content


In [12]:
!kaggle datasets download -d jeromeblanchet/commonsenseqa-nlp-dataset

  0% 0.00/2.05M [00:00<?, ?B/s]
100% 2.05M/2.05M [00:00<00:00, 68.0MB/s]


In [13]:
!ls

commonsenseqa-nlp-dataset.zip  toc.csv	      twcs
kaggle.json		       tut2-model.pt  tweets.tsv
sample.csv		       tut3-model.pt  Ubuntu-dialogue-corpus


In [14]:
!unzip \*.zip  && rm *.zip

Archive:  commonsenseqa-nlp-dataset.zip
  inflating: dev_rand_split.jsonl    
  inflating: dev_rand_split_EASY.jsonl  
  inflating: test_rand_split_no_answers.jsonl  
  inflating: train_rand_split.jsonl  
  inflating: train_rand_split_EASY.jsonl  


In [15]:
!ls

dev_rand_split_EASY.jsonl	  train_rand_split.jsonl
dev_rand_split.jsonl		  tut2-model.pt
kaggle.json			  tut3-model.pt
sample.csv			  twcs
test_rand_split_no_answers.jsonl  tweets.tsv
toc.csv				  Ubuntu-dialogue-corpus
train_rand_split_EASY.jsonl


In [22]:
import json
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize



In [34]:
data = []
with open('train_rand_split.jsonl', 'r') as f:
    for line in f:
        data.append(json.loads(line))
    extracted_data = json_normalize(data)
    extracted_data.columns = extracted_data.columns.map(lambda x: x.split(".")[-1])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


In [20]:
extracted_data.head()

,answerKey,id,question_concept,choices,stem
0,A,075e483d21c29a511267ef62bedc0461,punishing,"[{'label': 'A', 'text': 'ignore'}, {'label': '...",The sanctions against the school were a punish...
1,B,61fe6e879ff18686d7552425a36344c8,people,"[{'label': 'A', 'text': 'race track'}, {'label...",Sammy wanted to go to where the people were. ...
2,A,4c1cb0e95b99f72d55c068ba0255c54d,choker,"[{'label': 'A', 'text': 'jewelry store'}, {'la...",To locate a choker not located in a jewelry bo...
3,D,02e821a3e53cb320790950aab4489e85,highway,"[{'label': 'A', 'text': 'united states'}, {'la...",Google Maps and other highway and street GPS s...
4,C,23505889b94e880c3e89cff4ba119860,fox,"[{'label': 'A', 'text': 'pretty flowers.'}, {'...","The fox walked from the city into the forest, ..."


In [35]:
num_choices = len(extracted_data['choices'][0])

def get_choices(options, val):
    option = options[int(val)]
    return option.get('text')

choices = np.arange(num_choices)
choices = choices.astype('str')
#for c in choices:
#    extracted_data['choice_' + c] = extracted_data['choices'].apply(lambda x: get_choices(x, c))
answer_match = {'A':0, 'B':1, 'C':2, 'D':3, 'E':4}

def get_label(answer):
    return answer_match.get(answer)
extracted_data['label'] = extracted_data['answerKey'].apply(lambda x: get_label(x))

extracted_data.head()

,answerKey,id,question_concept,choices,stem,label
0,A,075e483d21c29a511267ef62bedc0461,punishing,"[{'label': 'A', 'text': 'ignore'}, {'label': '...",The sanctions against the school were a punish...,0
1,B,61fe6e879ff18686d7552425a36344c8,people,"[{'label': 'A', 'text': 'race track'}, {'label...",Sammy wanted to go to where the people were. ...,1
2,A,4c1cb0e95b99f72d55c068ba0255c54d,choker,"[{'label': 'A', 'text': 'jewelry store'}, {'la...",To locate a choker not located in a jewelry bo...,0
3,D,02e821a3e53cb320790950aab4489e85,highway,"[{'label': 'A', 'text': 'united states'}, {'la...",Google Maps and other highway and street GPS s...,3
4,C,23505889b94e880c3e89cff4ba119860,fox,"[{'label': 'A', 'text': 'pretty flowers.'}, {'...","The fox walked from the city into the forest, ...",2


In [29]:
extracted_data['choices'][0]

[{'label': 'A', 'text': 'ignore'},
 {'label': 'B', 'text': 'enforce'},
 {'label': 'C', 'text': 'authoritarian'},
 {'label': 'D', 'text': 'yell at'},
 {'label': 'E', 'text': 'avoid'}]

In [31]:
s = extracted_data['choices'][1]
l = extracted_data['label'][1]
print(get_choices(s, l))

populated areas


In [36]:
extracted_data['answer'] = extracted_data.apply(lambda x: get_choices(x['choices'], x['label']), axis=1)

extracted_data.head()

,answerKey,id,question_concept,choices,stem,label,answer
0,A,075e483d21c29a511267ef62bedc0461,punishing,"[{'label': 'A', 'text': 'ignore'}, {'label': '...",The sanctions against the school were a punish...,0,ignore
1,B,61fe6e879ff18686d7552425a36344c8,people,"[{'label': 'A', 'text': 'race track'}, {'label...",Sammy wanted to go to where the people were. ...,1,populated areas
2,A,4c1cb0e95b99f72d55c068ba0255c54d,choker,"[{'label': 'A', 'text': 'jewelry store'}, {'la...",To locate a choker not located in a jewelry bo...,0,jewelry store
3,D,02e821a3e53cb320790950aab4489e85,highway,"[{'label': 'A', 'text': 'united states'}, {'la...",Google Maps and other highway and street GPS s...,3,atlas
4,C,23505889b94e880c3e89cff4ba119860,fox,"[{'label': 'A', 'text': 'pretty flowers.'}, {'...","The fox walked from the city into the forest, ...",2,natural habitat


In [37]:
extracted_data.shape

(9741, 7)

In [45]:
qa = extracted_data[['stem', 'answer']]
qa.columns = ['question', 'answer']

In [46]:
# Save Q&A for later use
qa.to_csv("commonsenseqa.tsv", index=False, sep='\t')

In [38]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.data import Field, BucketIterator, TabularDataset

import spacy
import numpy as np

import random
import math
import time

In [47]:
# Use same field for both columns since they have a shared vocabulary
QA = Field(
    tokenize='spacy',
    lower=True,
    init_token='<sos>', 
    eos_token='<eos>'
)

fields = [('src', QA), ('trg', QA)]

In [48]:
# Create dataset
dataset = TabularDataset(
    path='commonsenseqa.tsv',
    format='TSV',
    fields=fields,
    skip_header=True
)

In [49]:
# Train/val split
train_data, valid_data, test_data = dataset.split(split_ratio=[0.75, 0.15, 0.1])
print(len(train_data), len(valid_data), len(test_data))

7306 974 1461


In [50]:
print(vars(train_data.examples[0]))

{'src': ['what', 'kind', 'of', 'radio', 'does', 'someone', 'use', 'to', 'reach', 'base', '?'], 'trg': ['transistor']}


In [51]:
#Build the vocabulary.
QA.build_vocab(train_data, min_freq = 2)

In [52]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [53]:
device

device(type='cuda')

In [54]:
BATCH_SIZE = 128

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
    sort_key = lambda x : len(x.src), 
    batch_size = BATCH_SIZE, 
    device = device)

In [55]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, dropout):
        super().__init__()

        self.hid_dim = hid_dim
        
        self.embedding = nn.Embedding(input_dim, emb_dim) #no dropout as only one layer!
        
        self.rnn = nn.GRU(emb_dim, hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src len, batch size, emb dim]
        
        outputs, hidden = self.rnn(embedded) #no cell state!
        
        #outputs = [src len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        
        #outputs are always from the top hidden layer
        
        return hidden

In [56]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, dropout):
        super().__init__()

        self.hid_dim = hid_dim
        self.output_dim = output_dim
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.GRU(emb_dim + hid_dim, hid_dim)
        
        self.fc_out = nn.Linear(emb_dim + hid_dim * 2, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, context):
        
        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #context = [n layers * n directions, batch size, hid dim]
        
        #n layers and n directions in the decoder will both always be 1, therefore:
        #hidden = [1, batch size, hid dim]
        #context = [1, batch size, hid dim]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
                
        emb_con = torch.cat((embedded, context), dim = 2)
            
        #emb_con = [1, batch size, emb dim + hid dim]
            
        output, hidden = self.rnn(emb_con, hidden)
        
        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        
        #seq len, n layers and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        
        output = torch.cat((embedded.squeeze(0), hidden.squeeze(0), context.squeeze(0)), 
                           dim = 1)
        
        #output = [batch size, emb dim + hid dim * 2]
        
        prediction = self.fc_out(output)
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden

In [57]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #last hidden state of the encoder is the context
        context = self.encoder(src)
        
        #context also used as the initial hidden state of the decoder
        hidden = context
        
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden state and the context state
            #receive output tensor (predictions) and new hidden state
            output, hidden = self.decoder(input, hidden, context)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1

        return outputs

In [58]:
INPUT_DIM = len(QA.vocab)
OUTPUT_DIM = len(QA.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, DEC_DROPOUT)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = Seq2Seq(enc, dec, device).to(device)

In [59]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.normal_(param.data, mean=0, std=0.01)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(4612, 256)
    (rnn): GRU(256, 512)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(4612, 256)
    (rnn): GRU(768, 512)
    (fc_out): Linear(in_features=1280, out_features=4612, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [60]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 11,421,188 trainable parameters


In [61]:
optimizer = optim.Adam(model.parameters())

In [64]:
PAD_IDX = QA.vocab.stoi[QA.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = PAD_IDX)

In [65]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.src
        trg = batch.trg
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [66]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.src
            trg = batch.trg

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [67]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [68]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 0m 2s
	Train Loss: 5.124 | Train PPL: 168.033
	 Val. Loss: 4.375 |  Val. PPL:  79.401
Epoch: 02 | Time: 0m 2s
	Train Loss: 4.329 | Train PPL:  75.839
	 Val. Loss: 4.357 |  Val. PPL:  78.056
Epoch: 03 | Time: 0m 2s
	Train Loss: 4.246 | Train PPL:  69.857
	 Val. Loss: 4.345 |  Val. PPL:  77.083
Epoch: 04 | Time: 0m 2s
	Train Loss: 4.194 | Train PPL:  66.319
	 Val. Loss: 4.371 |  Val. PPL:  79.122
Epoch: 05 | Time: 0m 2s
	Train Loss: 4.149 | Train PPL:  63.365
	 Val. Loss: 4.371 |  Val. PPL:  79.133
Epoch: 06 | Time: 0m 2s
	Train Loss: 4.105 | Train PPL:  60.670
	 Val. Loss: 4.394 |  Val. PPL:  80.969
Epoch: 07 | Time: 0m 2s
	Train Loss: 4.094 | Train PPL:  60.001
	 Val. Loss: 4.384 |  Val. PPL:  80.124
Epoch: 08 | Time: 0m 1s
	Train Loss: 4.051 | Train PPL:  57.483
	 Val. Loss: 4.401 |  Val. PPL:  81.522
Epoch: 09 | Time: 0m 2s
	Train Loss: 4.049 | Train PPL:  57.340
	 Val. Loss: 4.406 |  Val. PPL:  81.926
Epoch: 10 | Time: 0m 2s
	Train Loss: 4.068 | Train PPL:  58.422
